# RAG for Table Comparisons with LlamaParse + LlamaIndex

<a href="https://colab.research.google.com/github/run-llama/llama_parse/blob/main/examples/demo_table_comparisons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows you how to do comparisons across both tabular and text data across multiple PDF documents.

We load in multiple PDFs with embedded tables (2021 and 2020 10K filings for Apple) using LlamaParse, parse each into a hierarchy of tables/text objects, define a recursive retriever over each, and then compose both with a SubQuestionQueryEngine.

## Setup

Install core packages, download files, parse documents.

In [ ]:
!pip install llama-index
!pip install llama-index-core
!pip install llama-index-embeddings-openai
!pip install llama-index-question-gen-openai
!pip install llama-index-postprocessor-flag-embedding-reranker
!pip install git+https://github.com/FlagOpen/FlagEmbedding.git
!pip install llama-parse

In [ ]:
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2020/ar/_10-K-2020-(As-Filed).pdf" -O apple_2020_10k.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2021/q4/_10-K-2021-(As-Filed).pdf" -O apple_2021_10k.pdf

Some OpenAI and LlamaParse details

In [2]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os
# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-"

# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = "sk-"

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

embed_model=OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-3.5-turbo-0125")

Settings.llm = llm
Settings.embed_model = embed_model


## Using brand new `LlamaParse` PDF reader for PDF Parsing

we also compare two different retrieval/query engine strategies:
1. Using raw Markdown text as nodes for building index and apply simple query engine for generating the results;
2. Using `MarkdownElementNodeParser` for parsing the `LlamaParse` output Markdown results and building recursive retriever query engine for generation.

In [ ]:
from llama_parse import LlamaParse

docs_2021 = LlamaParse(result_type="markdown").load_data('./apple_2021_10k.pdf')
docs_2020 = LlamaParse(result_type="markdown").load_data('./apple_2020_10k.pdf')

## Create Recursive Retriever over each Document

We define a function to get a recursive retriever from each document. The steps are the following:
- Hierarchically parse the document using our `MarkdownElementNodeParser`, which will embed/summarize embedded tables.
- Load into a vector store. Under the hood we will automatically store links between nodes (e.g. table summary to table text).
- Get a query engine over the vector store, which performs retrieval/synthesis. Under the hood we will automatically perform recursive retrieval if there are links.

In [36]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo-0125"), num_workers=8)

In [46]:
import pickle
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=5,
    model="BAAI/bge-reranker-large",
)

def create_query_engine_over_doc(docs, nodes_save_path=None):
    """Big function to go from document path -> recursive retriever."""
    if nodes_save_path is not None and os.path.exists(nodes_save_path):
        raw_nodes = pickle.load(open(nodes_save_path, "rb"))
    else:
        raw_nodes = node_parser.get_nodes_from_documents(docs)
        if nodes_save_path is not None:
            pickle.dump(raw_nodes, open(nodes_save_path, "wb"))

    base_nodes, objects = node_parser.get_nodes_and_objects(
        raw_nodes
    )

    ### Construct Retrievers
    # construct top-level vector index + query engine
    vector_index = VectorStoreIndex(nodes=base_nodes+objects)
    query_engine = vector_index.as_query_engine(
        similarity_top_k=15,
        node_postprocessors=[reranker]
    )
    return query_engine, base_nodes

In [ ]:
query_engine_2021, nodes_2021 = create_query_engine_over_doc(
    docs_2021, nodes_save_path="2021_nodes.pkl"
)
query_engine_2020, nodes_2020 = create_query_engine_over_doc(
    docs_2020, nodes_save_path="2020_nodes.pkl"
)

In [38]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine


# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine_2021,
        metadata=ToolMetadata(
            name="apple_2021_10k",
            description=(
                "Provides information about Apple financials for year 2021"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=query_engine_2020,
        metadata=ToolMetadata(
            name="apple_2020_10k",
            description=(
                "Provides information about Apple financials for year 2020"
            ),
        ),
    ),
]

sub_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    llm=llm,
    use_async=True,
)

## Try out Some Comparisons

In [41]:
response = sub_query_engine.query(
    "Can you compare and contrast the deferred assets and liabilities in 2021 with 2020?"
)

Generated 4 sub questions.
[apple_2021_10k] Q: What are the deferred assets in 2021?
[apple_2021_10k] Q: What are the deferred liabilities in 2021?
[apple_2020_10k] Q: What are the deferred assets in 2020?
[apple_2020_10k] Q: What are the deferred liabilities in 2020?
[apple_2021_10k] A: $7,200
[apple_2020_10k] A: $10,138
[apple_2021_10k] A: $25,176 million
[apple_2020_10k] A: $19,336


In [40]:
print(str(response))

In 2021, the deferred assets increased by $5,840 million compared to 2020, while the deferred liabilities decreased by $2,938 million in the same period.


In [42]:
response = sub_query_engine.query(
    "Can you compare and contrast the total number of RSUs in 2021 and 2020?"
)

Generated 2 sub questions.
[apple_2021_10k] Q: What is the total number of RSUs in Apple's 2021 financials?
[apple_2020_10k] Q: What is the total number of RSUs in Apple's 2020 financials?
[apple_2021_10k] A: The total number of RSUs in Apple's 2021 financials is 240,427.
[apple_2020_10k] A: The total number of RSUs in Apple's 2020 financials is 310,778.


In [44]:
response = sub_query_engine.query(
    "Can you compare and contrast the risk factors in 2021 vs. 2020?"
)

Generated 2 sub questions.
[apple_2021_10k] Q: What are the risk factors mentioned in the 2021 financial report of Apple?
[apple_2020_10k] Q: What are the risk factors mentioned in the 2020 financial report of Apple?
[apple_2021_10k] A: The risk factors mentioned in the 2021 financial report of Apple include risks related to COVID-19, macroeconomic and industry risks, political events, trade and international disputes, natural disasters, public health issues, industrial accidents, credit risk, fluctuations in foreign currency exchange rates, changes in tax rates and legislation, volatility in the price of the company's stock, and exposure to legal proceedings and claims.
[apple_2020_10k] A: The risk factors mentioned in the 2020 financial report of Apple include the impact of the COVID-19 pandemic on the company's business operations, financial condition, and stock price; global and regional economic conditions affecting demand for products and services; competition in global markets w

In [45]:
print(str(response))

The risk factors mentioned in the 2021 financial report of Apple include risks related to COVID-19, macroeconomic and industry risks, political events, trade and international disputes, natural disasters, public health issues, industrial accidents, credit risk, fluctuations in foreign currency exchange rates, changes in tax rates and legislation, volatility in the price of the company's stock, and exposure to legal proceedings and claims. In contrast, the risk factors mentioned in the 2020 financial report of Apple focused more on the impact of the COVID-19 pandemic on the company's business operations, financial condition, and stock price; global and regional economic conditions affecting demand for products and services; competition in global markets with rapid technological changes; potential disruptions in the supply chain due to industrial accidents or public health issues; information technology system failures or network disruptions affecting business operations; risks associate